
![Metabolic-network.jpeg](Metabolic-network.jpeg)

## For tests
- Add reversible reactions
- Add exchange of B


## Steps
- Make all reversible reactions into two irreversible reactions
- Construct constraints for standard medium
- For each KO condition, construct constraints
- For each biolog conditions, construct constraints
- Construct problem from constraints, 

## Questions
- Constraints from cobra model must be included in list. Since dual, do LB become UB ( ex_A > 0, -> ex_A <- 0)
- We can move certain contraints out to problem level "To overcome this issue, $r_{B}$ can be moved to the objective.". How do we do these for all conditions?
- Will z be the only vector that defines which reactions to keep?
- Where will background reaction references come from?
- Where will weights for reactions come from? Will them make non-trivial impact if set to 1?


### Steps for Constraint for KO
 1. need constraint to specify KO condition, so Z works for WT and KO across
 2. KO is setting ub=0
 3. Remove z_i from omega constraint if rxn i in ko 
     r2[ko_index] <= Omega
    
```python 
for KO, biolog condition
   separate R
   for biolog, nutrient index = exchange rxn
   for KO, for every rxn involving ko gene (ignore \"or's\")
          R[ko_index] <= Omega, lb=0
```

## Things to keep in mind
1. All reactions by default in CROP are irreversible.
2. If a reaction is reversible, it must be transformed into two irreversible reactions
3. There are bounds on growth rate, below which the phenotype is "not viable" and above which it is considered "viable" 
4. There are bounds on the rate-limiting nutrients of a particular condition.  For the primal problem, this is represented as the normal EX_glc_e <= 10, but in the dual problem, this is pushed up to the objective so that growth can be  "partially reduced", such as B\cdot r_{glucose}
5. The dual variable of the rate-limiting nutrients must be >= 0.



In [1]:
from cobra.util.array import create_stoichiometric_matrix, constraint_matrices
import cobra
import escher
import sys
import pandas as pd
import numpy as np
import cvxpy as cvx
from IPython.display import display
pd.options.display.float_format = '{:.01f}'.format
sys.path.append('../..')
from tests.test_crop import load_model

def generate_constraints_for_medium(media, reactions):
    return
    #params:
    # input media
    #returns:
    # constraint list


#model = cobra.io.load_json_model('ABC_model.json')
model = cobra.io.load_json_model('ABC_extended_model.json')
# model = load_model()
#cobra.io.save_json_model(model, 'ABC_extended.json')
stoich_matrix = create_stoichiometric_matrix(model, 'DataFrame')

def idx(name, axis=1, S = stoich_matrix):
    if axis == 1:
        return S.columns.get_loc(name)
    else:
        return S.index.get_loc(name)
stoich_matrix

Set parameter Username
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "leghorn"


,SRC_A_e,SRC_B_e,SINK_C_e,SINK_D_e,SRC_E_e,SINK_F_e,Aim,Bim,Cex,Dex,Eim,Fex,A_to_B,A_to_C,B_to_CF,BE_to_D,C_to_D
A_ext,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B_ext,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_ext,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D_ext,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E_ext,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
F_ext,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
A,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-2.0,-1.0,0.0
C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,-1.0
D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0


In [4]:

class CropModel(object):
    def __init__(self, model, biomass_index):
        self.model = model
        self.stoich_matrix = create_stoichiometric_matrix(self.model, 'DataFrame')
        self.big_s = stoich_matrix.values
        self.n_metabolites, self.n_reactions = stoich_matrix.shape
        self.biomass = idx(biomass_index)

        # global variables
        self.c = np.zeros(self.n_reactions)
        self.c[self.biomass] = 1
        self.z = cvx.Variable(self.n_reactions, boolean=True)
        self.m = cvx.Variable(self.n_metabolites)
        self.weights = np.ones(self.n_reactions)*0.5

        # objective to be used
        self.objective = None
        self.constraints = []

        # no grow conditions
        self.no_growth_variables = []

        # grow condition variables
        self.growth_variables = []


def gather_for_no_growth(crop_model, no_grow_swap_index):
    r = cvx.Variable(crop_model.n_reactions, name=f'r_{len(crop_model.no_growth_variables)}')
    omega = 1000
    B = 10
    constraints = [
        crop_model.big_s@r    == 0,
        crop_model.big_s.T@crop_model.m - r       == -1*crop_model.c,
        r[:no_grow_swap_index]     <= omega*np.eye(no_grow_swap_index)@(1 - crop_model.z[:no_grow_swap_index]),
        r[no_grow_swap_index]      >=0,
        r[no_grow_swap_index+1:]   <= omega*np.eye(crop_model.n_reactions - no_grow_swap_index - 1)@(1 - crop_model.z[no_grow_swap_index + 1:])
    ]
    obj_function = B*r[no_grow_swap_index]

    crop_model.no_growth_variables.append(r)
    return obj_function, constraints


def generate_constraints_no_growth_condition(model, model_modifications, base_media, no_growth):
    # generate no-growth conditions
    for condition in no_growth:
        # generate r_i, o_i, constraints
        local_obj, constraints = gather_for_no_growth(model, condition)
        if model.objective is None:
            model.objective = local_obj
        else:
            model.objective += local_obj
        model.constraints += constraints


def gather_for_growth(crop_model):
#     u = np.eye(crop_model.n_reactions)*10.0
    w = cvx.Variable(crop_model.n_reactions, name=f'w_{len(crop_model.growth_variables)}')
    constraints = [
        crop_model.big_s@w == 0,
        0 >= w,
        w[crop_model.biomass] >= minimal_growth
    ]
#     crop_model.growth_variables.append(u)
    crop_model.growth_variables.append(w)
    return constraints


def generate_constraints_growth_condition(model, model_modifications=None, base_media=None, growth_condition=None):
    # generate growth conditions
    model.constraints += gather_for_growth(model)



In [5]:
# global variables
minimal_growth  = 0.1
atp_maintenance = 0.1


obj_index = 'SINK_D_e'
crop_example = CropModel(model, biomass_index=obj_index)

no_growth_conditions = [idx('SRC_B_e')]

generate_constraints_no_growth_condition(
    crop_example,
    model_modifications = None, 
    base_media = None, 
    no_growth=no_growth_conditions
)

generate_constraints_growth_condition(
    crop_example,
    model_modifications = None, 
    base_media = None, 

)

objective = crop_example.objective
wt_conditions = crop_example.constraints
# wt_conditions = []


problem = cvx.Problem(
    cvx.Minimize(
        objective
    ),
    crop_example.constraints
)

problem.solve(solver=cvx.SCIPY, verbose=True )

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 13 01:44:49 PM: Your problem has 63 variables, 8 constraints, and 0 parameters.
(CVXPY) Dec 13 01:44:49 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 13 01:44:49 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 13 01:44:49 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 13 01:44:49 PM: Compiling problem (target solver=SCIPY).
(CVXPY) Dec 13 01:44:49 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -

inf

In [6]:
for i in crop_example.no_growth_variables:
    print(i)
for i in crop_example.growth_variables:
    print(i)

r_0
w_0


In [10]:
reactions = pd.DataFrame( crop_example.z.value, index=crop_example.model.reactions, columns=['z'])
for n, var in enumerate(crop_example.no_growth_variables):
    print(n, var, var.value)
    reactions[f'r_{n}'] = pd.Series(np.squeeze(np.asarray(var.value)), index=crop_example.model.reactions)

# reactions['r'] = pd.Series(np.squeeze(np.asarray(r.value)),index=rxns)
# reactions['w'] = pd.Series(np.squeeze(np.asarray(w.value)),index=rxns)
metabolites = pd.DataFrame(crop_example.m.value, index=crop_example.model.metabolites, columns=['m'])
display(reactions)

display(metabolites)

0 r_0 None


,z,r_0
SRC_A_e: <=> A_ext,NaN,None
SRC_B_e: <=> B_ext,NaN,None
SINK_C_e: C_ext <=>,NaN,None
SINK_D_e: D_ext <=>,NaN,None
SRC_E_e: <=> E_ext,NaN,None
SINK_F_e: F_ext <=>,NaN,None
Aim: A_ext <=> A,NaN,None
Bim: B_ext <=> B,NaN,None
Cex: C <=> C_ext,NaN,None
Dex: D <=> D_ext,NaN,None


,m
A_ext,NaN
B_ext,NaN
C_ext,NaN
D_ext,NaN
E_ext,NaN
F_ext,NaN
A,NaN
B,NaN
C,NaN
D,NaN


In [ ]:
#example inputs needed
"""
args = dict(
    model,
    media,
    modifications = (
        added_reactions, # genes via reactions
        removed_reactions, # genes via reactions
    ),
    growth=bool
)
"""